In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
# trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
# devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
# testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


12152
2867
3005


In [4]:
print(testSentences[3])

[['Die', 'O'], ['sechs', 'O'], ['Abgeordneten', 'O'], ['der', 'O'], ['Grünen', 'B-ORG'], [',', 'O'], ['die', 'O'], ['seit', 'O'], ['Bildung', 'O'], ['einer', 'O'], ['Großen', 'O'], ['Koalition', 'O'], ['von', 'O'], ['Christ-', 'O'], ['und', 'O'], ['Sozialdemokraten', 'O'], ['die', 'O'], ['Opposition', 'O'], ['darstellen', 'O'], [',', 'O'], ['hatten', 'O'], ['sich', 'O'], ['beim', 'O'], ['Vorsteher', 'O'], ['Rainer', 'B-PER'], ['Bergert', 'I-PER'], ['für', 'O'], ['ihr', 'O'], ['Fehlen', 'O'], ['entschuldigen', 'O'], ['lassen', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0
n_gt_100 = 0
for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)
        if len(sentence) > 100:
            n_gt_100 += 1
print(n_gt_100)

0


In [6]:
print(len(labelSet))
print(models.max_sequence_length)

9
100


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'B-MISC': 3, 'O': 1, 'B-LOC': 2, 'I-ORG': 7, 'I-MISC': 4, 'I-PER': 6, 'PADDING_TOKEN': 0, 'B-ORG': 9, 'I-LOC': 8, 'B-PER': 5}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'mainly_numeric': 6, 'numeric': 1, 'PADDING_TOKEN': 0, 'allLower': 2, 'contains_digit': 7, 'other': 5, 'allUpper': 3, 'initialUpper': 4}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'ö': 54, 'ä': 55, 'J': 29, 'PADDING_TOKEN': 0, 'I': 5, 'o': 61, 'è': 101, 'Z': 6, 'P': 7, '!': 8, 'Ä': 57, '&': 9, 'Y': 58, 'UNKNOWN': 103, 'd': 60, 'S': 10, 'á': 63, '</S>': 2, 'l': 64, '=': 32, 'ê': 11, 'ß': 12, '-': 66, 'b': 14, 'h': 67, 'V': 68, 'x': 34, '5': 15, 'j': 70, 'Ü': 71, 'r': 97, 'M': 72, 't': 17, '</W>': 4, 'C': 65, 'H': 18, 'f': 73, '<S>': 1, '.': 19, 'ç': 20, 'w': 23, 'W': 21, '/': 99, 'g': 22, '9': 75, 'z': 79, '2': 25, '8': 76, 'N': 26, 'ñ': 28, '+': 77, 'B': 56, '0': 78, 'p': 30, 'í': 31, '<W>': 3, 'i': 33, '"': 35, 'R': 24, '§': 83, 'L': 81, '7': 82, ')': 36, ':': 37, 'A': 38, 'q': 84, 'E': 85, 'v': 87, 'a': 88, '4': 98, 'n': 89, 'D': 69, '6': 74, 'm': 39, 'é': 80, "'": 62, 'F': 40, 'ì': 41, 'T': 90, 'U': 91, 'ô': 59, '*': 42, 'y': 27, 's': 43, 'ó': 16, 'X': 44, 'O': 86, 'G': 92, 'e': 93, 'ü': 94, '%': 95, ';': 45, 'Q': 96, '1': 46, 'u': 13, '?': 47, '(': 48, 'K': 49, 'k': 50, ',': 100, 'ò': 51, '3': 102, 'Ö': 52, 'c': 53}


In [14]:
print(trainSentences[0]) 

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
5


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

10
10


# Test Model

In [ ]:
# import importlib
# importlib.reload(models)

In [ ]:
tmp_model_filename = 'model_lstm_conll.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_lstm_v2()

In [ ]:
#import importlib
#importlib.reload(utils)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm_conll.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'model_lstm_conll.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_lstm_v2()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=256), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 3328        char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case

Epoch 1/10
379/380 [============================>.] - ETA: 0s - loss: 0.0372 - acc: 0.9903
New maximum F1 score: 0.7762483130904183 (before: 0) Saving to model_lstm_conll.1.h5
380/380 [==============================] - 256s 674ms/step - loss: 0.0371 - acc: 0.9903 - val_loss: 0.0212 - val_acc: 0.9943
Epoch 2/10
379/380 [============================>.] - ETA: 0s - loss: 0.0136 - acc: 0.9959
New maximum F1 score: 0.8085969180859691 (before: 0.7762483130904183) Saving to model_lstm_conll.1.h5
380/380 [==============================] - 249s 655ms/step - loss: 0.0136 - acc: 0.9959 - val_loss: 0.0154 - val_acc: 0.9950
Epoch 3/10
379/380 [============================>.] - ETA: 0s - loss: 0.0082 - acc: 0.9966
New maximum F1 score: 0.8180581996192547 (before: 0.8085969180859691) Saving to model_lstm_conll.1.h5
380/380 [==============================] - 249s 655ms/step - loss: 0.0082 - acc: 0.9966 - val_loss: 0.0109 - val_acc: 0.9953
Epoch 4/10
379/380 [============================>.] - ETA: 0s -

Epoch 5/10
379/380 [============================>.] - ETA: 0s - loss: -0.0027 - acc: 0.9975
New maximum F1 score: 0.8399206873760741 (before: 0.8264551687054712) Saving to model_lstm_conll.2.h5
380/380 [==============================] - 251s 659ms/step - loss: -0.0028 - acc: 0.9975 - val_loss: 9.6199e-04 - val_acc: 0.9957
Epoch 6/10
379/380 [============================>.] - ETA: 0s - loss: -0.0080 - acc: 0.9977
New maximum F1 score: 0.8434978171715836 (before: 0.8399206873760741) Saving to model_lstm_conll.2.h5
380/380 [==============================] - 251s 662ms/step - loss: -0.0080 - acc: 0.9977 - val_loss: -0.0047 - val_acc: 0.9958
Epoch 7/10
379/380 [============================>.] - ETA: 0s - loss: -0.0139 - acc: 0.9980
New maximum F1 score: 0.8457815565729234 (before: 0.8434978171715836) Saving to model_lstm_conll.2.h5
380/380 [==============================] - 252s 663ms/step - loss: -0.0139 - acc: 0.9980 - val_loss: -0.0100 - val_acc: 0.9959
Epoch 8/10
380/380 [==============

Epoch 10/10
379/380 [============================>.] - ETA: 0s - loss: -0.0314 - acc: 0.9987
New maximum F1 score: 0.8516060525617203 (before: 0.8470744680851064) Saving to model_lstm_conll.3.h5
380/380 [==============================] - 251s 662ms/step - loss: -0.0315 - acc: 0.9987 - val_loss: -0.0262 - val_acc: 0.9959
Epoch 1/10
48/48 [==============================] - 108s 2s/step - loss: -0.0360 - acc: 0.9989 - val_loss: -0.0271 - val_acc: 0.9960
Epoch 2/10
47/48 [============================>.] - ETA: 0s - loss: -0.0371 - acc: 0.9990
New maximum F1 score: 0.8525490196078431 (before: 0.8516060525617203) Saving to model_lstm_conll.3.h5
48/48 [==============================] - 88s 2s/step - loss: -0.0371 - acc: 0.9990 - val_loss: -0.0279 - val_acc: 0.9960
Epoch 3/10
48/48 [==============================] - 91s 2s/step - loss: -0.0380 - acc: 0.9991 - val_loss: -0.0286 - val_acc: 0.9960
Epoch 4/10
47/48 [============================>.] - ETA: 0s - loss: -0.0389 - acc: 0.9991
New maximu

Epoch 3/10
47/48 [============================>.] - ETA: 0s - loss: -0.0184 - acc: 0.9986
New maximum F1 score: 0.8511873350923482 (before: 0.8494708994708996) Saving to model_lstm_conll.4.h5
48/48 [==============================] - 89s 2s/step - loss: -0.0184 - acc: 0.9986 - val_loss: -0.0106 - val_acc: 0.9960
Epoch 4/10
48/48 [==============================] - 90s 2s/step - loss: -0.0194 - acc: 0.9988 - val_loss: -0.0111 - val_acc: 0.9960
Epoch 5/10
48/48 [==============================] - 88s 2s/step - loss: -0.0204 - acc: 0.9987 - val_loss: -0.0121 - val_acc: 0.9960
Epoch 6/10
47/48 [============================>.] - ETA: 0s - loss: -0.0213 - acc: 0.9988
New maximum F1 score: 0.8530843088254637 (before: 0.8511873350923482) Saving to model_lstm_conll.4.h5
48/48 [==============================] - 89s 2s/step - loss: -0.0213 - acc: 0.9988 - val_loss: -0.0130 - val_acc: 0.9960
Epoch 7/10
48/48 [==============================] - 89s 2s/step - loss: -0.0224 - acc: 0.9988 - val_loss: -0.0

# Save final model

In [19]:
import shutil, json
# copy file for best run
shutil.copyfile('model_lstm_conll.9.h5', '../models/final_model_conll.h5')
shutil.copyfile('model_lstm_conll.9.h5.indexes', '../models/final_model_conll.indexes')

'../models/final_model_conll.indexes'

In [20]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_conll.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

(0.8722241202596516, 0.8370491803278689, 0.8542747197590764)


In [22]:
f = open('conll_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = " ".join([tok[0], correctlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close

<function TextIOWrapper.close()>